<a href="https://colab.research.google.com/github/PatrickAllenCooper/unsupervised-causal-association/blob/main/vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VAE (Generic Implementation)

In [ ]:
# we begin with an elementary vae as the baseline
from pl_bolts.models.autoencoders import VAE

model = VAE()
trainer = Trainer()
trainer.fit(model)

from pl_bolts.modsels.autoencoders import VAE

vae = VAE(input_height=32)
print(VAE.pretrained_weights_available())
vae = vae.from_pretrained('cifar10-resnet18')

vae.freeze()